# * Mobile Market Share from Meta

In [1]:
import os
import sys
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import re

curr_dt = dt.datetime.now().date()
next_week_dt = curr_dt + dt.timedelta(days=7)
str_curr_dt = curr_dt.strftime('%Y%m%d')
str_next_week_dt = next_week_dt.strftime('%Y%m%d')
yyyymm_curr = curr_dt.strftime('%Y%m')
yyyymm_next_week = next_week_dt.strftime('%Y%m')

### Sample data source

In [32]:
src_file_age = 'data/MB MKS by province(Age)_2025-12-28_All.csv'
df_age = pd.read_csv(src_file_age, skiprows=2)
df_age.head(3)

,Provinces & Age,TrueMove H-Mobile Market share (normalized),AIS-Mobile Market share (normalized),dtac-Mobile Market share (normalized),TOT3G-Mobile Market share (normalized),TrueMove H-Mobile Market size,AIS-Mobile Market size,dtac-Mobile Market size,TOT3G-Mobile Market size,TrueMove H-Mobile Total customers,AIS-Mobile Total customers,dtac-Mobile Total customers,TOT3G-Mobile Total customers
0,"Changwat Chai Nat, 13-17",37.19,50.20,12.61,--,"11,000","11,000","11,000","11,000","4,700","6,100","1,600",--
1,"Changwat Chai Nat, 18-24",38.36,46.81,14.83,--,"31,000","31,000","31,000","31,000","13,000","16,000","5,100",--
2,"Changwat Chai Nat, 25-34",37.59,49.03,13.38,--,"67,000","67,000","67,000","67,000","26,000","34,000","9,300",--


In [28]:
src_file_connect = 'data/MB MKS by province(Connection)_2025-12-28_All.csv'
df_connect = pd.read_csv(src_file_connect, skiprows=2)
df_connect.head(3)

,Provinces & Connection type,TrueMove H-Mobile Market share (normalized),AIS-Mobile Market share (normalized),dtac-Mobile Market share (normalized),TOT3G-Mobile Market share (normalized),TrueMove H-Mobile Market size,AIS-Mobile Market size,dtac-Mobile Market size,TOT3G-Mobile Market size,TrueMove H-Mobile Total customers,AIS-Mobile Total customers,dtac-Mobile Total customers,TOT3G-Mobile Total customers
0,"Changwat Chai Nat, 2G",--,--,--,--,--,--,--,--,--,--,--,--
1,"Changwat Chai Nat, 3G",27.51,60.14,12.35,--,"4,000","4,000","4,000","4,000","1,100","2,600",460,--
2,"Changwat Chai Nat, 4G",30.28,56.15,13.57,--,"125,000","125,000","125,000","125,000","39,000","71,000","18,000",--


In [29]:
src_file_device = 'data/MB MKS by province(Device)_2025-12-28_All.csv'
df_device = pd.read_csv(src_file_device, skiprows=2)
df_device.head(3)

,Provinces & Device price (USD),TrueMove H-Mobile Market share (normalized),AIS-Mobile Market share (normalized),dtac-Mobile Market share (normalized),TOT3G-Mobile Market share (normalized),TrueMove H-Mobile Market size,AIS-Mobile Market size,dtac-Mobile Market size,TOT3G-Mobile Market size,TrueMove H-Mobile Total customers,AIS-Mobile Total customers,dtac-Mobile Total customers,TOT3G-Mobile Total customers
0,"Changwat Chai Nat, Price unknown",38.93,48.83,12.24,--,"7,200","7,200","7,200","7,200","2,900","3,600","1,000",--
1,"Changwat Chai Nat, $1,000 or more",40.10,46.67,13.23,--,"36,000","36,000","36,000","36,000","15,000","18,000","5,100",--
2,"Changwat Chai Nat, Less than $100",29.66,58.08,12.26,--,"11,000","11,000","11,000","11,000","3,200","6,400","1,400",--


In [30]:
src_file_gender = 'data/MB MKS by province(Gender)_2025-12-28_All.csv'
df_gender = pd.read_csv(src_file_gender, skiprows=2)
df_gender.head(3)

,Provinces & Gender,TrueMove H-Mobile Market share (normalized),AIS-Mobile Market share (normalized),dtac-Mobile Market share (normalized),TOT3G-Mobile Market share (normalized),TrueMove H-Mobile Market size,AIS-Mobile Market size,dtac-Mobile Market size,TOT3G-Mobile Market size,TrueMove H-Mobile Total customers,AIS-Mobile Total customers,dtac-Mobile Total customers,TOT3G-Mobile Total customers
0,"Changwat Chai Nat, Female",34.65,51.59,13.76,--,"112,000","112,000","112,000","112,000","40,000","60,000","16,000",--
1,"Changwat Chai Nat, Male",33.26,53.48,13.27,--,"108,000","108,000","108,000","108,000","37,000","60,000","15,000",--
2,"Changwat Surin, Female",28.66,60.78,10.53,0.03,"396,000","396,000","396,000","396,000","119,000","252,000","45,000",--


### Step 1 : Import Data Source

In [2]:
# 1. Specify the path to your CSV files
src_path = './data' 

# 2. Use glob to match all csv files in that folder
all_files = glob.glob(os.path.join(src_path, "*.csv"))
all_files

# 3. Use a list comprehension to read each file and concat them
# df = pd.concat((pd.read_csv(f, skiprows=2) for f in all_files), ignore_index=True)

['./data\\MB MKS by province(Age)_2025-12-28_All.csv',
 './data\\MB MKS by province(Connection)_2025-12-28_All.csv',
 './data\\MB MKS by province(Device)_2025-12-28_All.csv',
 './data\\MB MKS by province(Gender)_2025-12-28_All.csv']

In [3]:
df = []

for f in all_files:

    # Read the file starting at row 3 (index 2)
    df_temp = pd.read_csv(f, skiprows=2)

    # Rename by referencing the first column index (0)
    df_temp = df_temp.rename(columns={df_temp.columns[0]: 'Provinces & Category'})

    # Optional: add a column to track which file the data came from
    df_temp['Source'] = os.path.basename(f)

    # Append to our list
    df.append(df_temp)

# 4. Combine everything into one big DataFrame
all_src_df = pd.concat(df, ignore_index=True)

print(f"Successfully imported {len(all_files)} files.")

all_src_df.columns = ['Provinces & Category'
    , 'TMH-MB MKS', 'AIS-MB MKS', 'DTAC-MB MKS', 'TOT-MB MKS'
    , 'TMH-MB Market Size', 'AIS-MB Market Size', 'DTAC-MB Market Size', 'TOT-MB Market Size'
    , 'TMH-MB Total Customer', 'AIS-MB Total Customer', 'DTAC-MB Total Customer', 'TOT-MB Total Customer'
    , 'Source']

all_src_df.head(3)

Successfully imported 4 files.


,Provinces & Category,TMH-MB MKS,AIS-MB MKS,DTAC-MB MKS,TOT-MB MKS,TMH-MB Market Size,AIS-MB Market Size,DTAC-MB Market Size,TOT-MB Market Size,TMH-MB Total Customer,AIS-MB Total Customer,DTAC-MB Total Customer,TOT-MB Total Customer,Source
0,"Changwat Chai Nat, 13-17",37.19,50.2,12.61,--,"11,000","11,000","11,000","11,000","4,700","6,100","1,600",--,MB MKS by province(Age)_2025-12-28_All.csv
1,"Changwat Chai Nat, 18-24",38.36,46.81,14.83,--,"31,000","31,000","31,000","31,000","13,000","16,000","5,100",--,MB MKS by province(Age)_2025-12-28_All.csv
2,"Changwat Chai Nat, 25-34",37.59,49.03,13.38,--,"67,000","67,000","67,000","67,000","26,000","34,000","9,300",--,MB MKS by province(Age)_2025-12-28_All.csv


In [4]:
# Use expand=True to turn the result into a DataFrame
new_cols = all_src_df['Provinces & Category'].str.split(', ', expand=True)

# Assign those back to your main DataFrame
all_src_df[['Provinces', 'Category']] = new_cols

all_src_df.head(3)

,Provinces & Category,TMH-MB MKS,AIS-MB MKS,DTAC-MB MKS,TOT-MB MKS,TMH-MB Market Size,AIS-MB Market Size,DTAC-MB Market Size,TOT-MB Market Size,TMH-MB Total Customer,AIS-MB Total Customer,DTAC-MB Total Customer,TOT-MB Total Customer,Source,Provinces,Category
0,"Changwat Chai Nat, 13-17",37.19,50.2,12.61,--,"11,000","11,000","11,000","11,000","4,700","6,100","1,600",--,MB MKS by province(Age)_2025-12-28_All.csv,Changwat Chai Nat,13-17
1,"Changwat Chai Nat, 18-24",38.36,46.81,14.83,--,"31,000","31,000","31,000","31,000","13,000","16,000","5,100",--,MB MKS by province(Age)_2025-12-28_All.csv,Changwat Chai Nat,18-24
2,"Changwat Chai Nat, 25-34",37.59,49.03,13.38,--,"67,000","67,000","67,000","67,000","26,000","34,000","9,300",--,MB MKS by province(Age)_2025-12-28_All.csv,Changwat Chai Nat,25-34


In [6]:
def metric_group(x):

    x_input = x
    metric = ''

    if re.search('Age', x_input): metric = 'Age'
    elif re.search('Connection', x_input): metric = 'Connection'
    elif re.search('Device', x_input): metric = 'Device'
    elif re.search('Gender', x_input): metric = 'Gender'

    return metric

# Pass the function name (without parentheses) to .apply()
all_src_df['Metric'] = all_src_df['Source'].apply(metric_group)
all_src_df#.head(3)

,Provinces & Category,TMH-MB MKS,AIS-MB MKS,DTAC-MB MKS,TOT-MB MKS,TMH-MB Market Size,AIS-MB Market Size,DTAC-MB Market Size,TOT-MB Market Size,TMH-MB Total Customer,AIS-MB Total Customer,DTAC-MB Total Customer,TOT-MB Total Customer,Source,Provinces,Category,Metric
0,"Changwat Chai Nat, 13-17",37.19,50.2,12.61,--,"11,000","11,000","11,000","11,000","4,700","6,100","1,600",--,MB MKS by province(Age)_2025-12-28_All.csv,Changwat Chai Nat,13-17,Age
1,"Changwat Chai Nat, 18-24",38.36,46.81,14.83,--,"31,000","31,000","31,000","31,000","13,000","16,000","5,100",--,MB MKS by province(Age)_2025-12-28_All.csv,Changwat Chai Nat,18-24,Age
2,"Changwat Chai Nat, 25-34",37.59,49.03,13.38,--,"67,000","67,000","67,000","67,000","26,000","34,000","9,300",--,MB MKS by province(Age)_2025-12-28_All.csv,Changwat Chai Nat,25-34,Age
3,"Changwat Chai Nat, 35-44",35.03,52.19,12.78,--,"43,000","43,000","43,000","43,000","15,000","22,000","5,500",--,MB MKS by province(Age)_2025-12-28_All.csv,Changwat Chai Nat,35-44,Age
4,"Changwat Chai Nat, 45-54",30.87,55.07,14.06,--,"30,000","30,000","30,000","30,000","9,600","17,000","4,200",--,MB MKS by province(Age)_2025-12-28_All.csv,Changwat Chai Nat,45-54,Age
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1766,"Changwat Phatthalung, Male",23.3,64.83,11.81,0.06,"163,000","163,000","163,000","163,000","41,000","110,000","20,000",--,MB MKS by province(Gender)_2025-12-28_All.csv,Changwat Phatthalung,Male,Gender
1767,"Changwat Nakhon Sawan, Female",28.47,53.96,17.57,--,"355,000","355,000","355,000","355,000","107,000","201,000","66,000",--,MB MKS by province(Gender)_2025-12-28_All.csv,Changwat Nakhon Sawan,Female,Gender
1768,"Changwat Nakhon Sawan, Male",29.23,54.82,15.95,--,"332,000","332,000","332,000","332,000","102,000","191,000","55,000",--,MB MKS by province(Gender)_2025-12-28_All.csv,Changwat Nakhon Sawan,Male,Gender
1769,"Changwat Mae Hong Son, Female",38.93,55.18,5.9,--,"79,000","79,000","79,000","79,000","33,000","48,000","5,100",--,MB MKS by province(Gender)_2025-12-28_All.csv,Changwat Mae Hong Son,Female,Gender


### Step 2 : Export Result

In [11]:
''' Generate CSV file '''

op_dir = 'output'
op_file = f'mobile_mks_province_{str_curr_dt}'

all_src_df.to_csv(f'{op_dir}/{op_file}.csv', index=False, encoding='utf-8')
print(f'\nGenerate "{op_file}.csv" successfully')


Generate "mobile_mks_province_20260129.csv" successfully
